In [1]:
import optuna
import json
import pprint
from uuid import uuid4

# Third Party Library
import pandas as pd
from ex_utils.config.paths import get_dataset_path
from ex_utils.config.quality_metrics import qm_names
from ex_utils.share import (
    ex_path,
    generate_seed_median_df,
    generate_sscalers,
)
from ex_utils.utils.graph import (
    load_nx_graph,
    nx_graph_preprocessing,
)

# Standard Library
import argparse
from time import perf_counter

# Third Party Library
import matplotlib.pyplot as plt
import networkx as nx
import optuna
import pandas as pd
from egraph import Drawing, all_sources_bfs
from ex_utils.config.dataset import dataset_names
from ex_utils.config.paths import get_dataset_path
from ex_utils.config.quality_metrics import qm_names
from ex_utils.share import draw, ex_path, generate_base_df_data, rate2pivots
from ex_utils.utils.graph import (
    egraph_graph,
    load_nx_graph,
    nx_graph_preprocessing,
)
from tqdm import tqdm
import math


export_data = {}

EDGE_WEIGHT = 30
d_names = sorted(
    [
        "1138_bus",
        "USpowerGrid",
        "dwt_1005",
        "poli",
        "qh882",
    ]
)

seeds = list(range(15))
n_split = 10
multi_n_trials = 300
single_n_trials = 300

pref_array = [0.5, 0.5, 0.5, 5.0, 2.0, 0.5, 4.0, 0.5, 4.0, 0.5]
pref = dict(zip(qm_names, pref_array))
drawing_seeds = list(range(3))

params = dict(
    zip(
        d_names,
        [
            {"empirical": {"pivots": 50, "iterations": 100, "eps": 0.1}}
            for _ in range(len(d_names))
        ],
    )
)


def gen_pic(params, eg_graph, nx_graph, eg_indices, seed, path):
    path.parent.mkdir(parents=True, exist_ok=True)
    fig, axes = plt.subplots(
        figsize=(8, 8),
        dpi=300,
        facecolor="white",
        ncols=1,
        nrows=1,
        squeeze=False,
    )
    ax = axes[0][0]
    ax.set_aspect("equal")

    eg_drawing = Drawing.initial_placement(eg_graph)
    pos = draw(
        pivots=params["pivots"],
        iterations=params["iterations"],
        eps=params["eps"],
        eg_graph=eg_graph,
        eg_indices=eg_indices,
        eg_drawing=eg_drawing,
        edge_weight=EDGE_WEIGHT,
        seed=seed,
    )
    nx.draw(
        nx_graph,
        pos=pos,
        node_size=3,
        # node_color="#AB47BC",
        width=1,
        edge_color="#CFD8DC",
        ax=ax,
        margins=0,
    )

    # ax.set_axis_on()
    ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
    plt.tight_layout(pad=-1)
    plt.savefig(path)
    plt.close()


for d_name in d_names:
    export_data[d_name] = []
    df_paths = [
        ex_path.joinpath(
            f"data/grid/{d_name}/n_split={n_split}/seed={seed}.pkl"
        )
        for seed in seeds
    ]
    df = pd.concat([pd.read_pickle(path) for path in df_paths])
    df = generate_seed_median_df(df)
    scalers = generate_sscalers(df)

    dataset_path = get_dataset_path(d_name)
    nx_graph = nx_graph_preprocessing(
        load_nx_graph(dataset_path=dataset_path), EDGE_WEIGHT
    )
    p_max = max(1, int(len(nx_graph.nodes) * 0.25))
    eg_graph, eg_indices = egraph_graph(nx_graph=nx_graph)
    eg_distance_matrix = all_sources_bfs(eg_graph, EDGE_WEIGHT)

    multi_obj_study_name = f"multi-obj-{multi_n_trials}"
    multi_db_uri = f'sqlite:///{ex_path.joinpath(f"data/optimization/{d_name}-multi-obj.db")}'
    multi_obj_study = optuna.load_study(
        storage=multi_db_uri, study_name=multi_obj_study_name
    )
    pareto_front = list(multi_obj_study.best_trials)

    # multi
    multi = -float("inf")
    for pareto_optimal in pareto_front:
        pareto_optimal_quality_metrics = pareto_optimal.user_attrs[
            "median_quality_metrics"
        ]
        pareto_optimal_scaled_quality_metrics = dict(
            [
                (
                    qm_name,
                    scalers[qm_name].transform(
                        [[pareto_optimal_quality_metrics[qm_name]]]
                    )[0][0],
                )
                for qm_name in qm_names
            ]
        )
        pareto_optimal_scaled_quality_metrics_sum = sum(
            [
                pareto_optimal_scaled_quality_metrics[qm_name] * pref[qm_name]
                for qm_name in qm_names
            ]
        )

        if multi < pareto_optimal_scaled_quality_metrics_sum:
            multi = pareto_optimal_scaled_quality_metrics_sum
            params[d_name]["multi"] = pareto_optimal.params

    # single
    single_obj_study_name = f'single-obj_{",".join(map(str, pref_array))}'
    single_db_uri = f'sqlite:///{ex_path.joinpath(f"data/optimization/{d_name}-user-experiment.db")}'
    sinigle_obj_study = optuna.load_study(
        storage=single_db_uri, study_name=single_obj_study_name
    )

    params[d_name]["single"] = sinigle_obj_study.best_trial.params

    for d_seed in drawing_seeds:
        path = f"data/exex/{d_name.replace('_', '-')}/empirical-{d_seed}.png"
        gen_pic(
            params[d_name]["empirical"],
            eg_graph,
            nx_graph,
            eg_indices,
            d_seed,
            ex_path.joinpath(path),
        )

        path = f"data/exex/{d_name.replace('_', '-')}/multi-{d_seed}.png"
        gen_pic(
            params[d_name]["multi"],
            eg_graph,
            nx_graph,
            eg_indices,
            d_seed,
            ex_path.joinpath(path),
        )

        path = f"data/exex/{d_name.replace('_', '-')}/single-{d_seed}.png"
        gen_pic(
            params[d_name]["single"],
            eg_graph,
            nx_graph,
            eg_indices,
            d_seed,
            ex_path.joinpath(path),
        )

/Users/fuga_takata/dev/vdslab-project/hyperparameter-in-graph-drawing/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from random import shuffle

with open(ex_path.joinpath("data/exex/params.json"), mode="w") as f:
    json.dump(params, f)

In [3]:
from PIL import Image, ImageDraw, ImageFont


def vertical_concat(img_a, img_b):
    return vertical_concat_with_labels(img_a, "A", img_b, "B")


def vertical_concat_with_labels(img_a, label_a, img_b, label_b):
    dst = Image.new("RGB", (img_a.width, img_a.height + img_b.height))
    dst.paste(img_a, (0, 0))
    dst.paste(img_b, (0, img_a.height))
    draw = ImageDraw.Draw(dst)
    x = img_a.width // 20
    y = img_a.height // 20
    font = ImageFont.truetype(font="Arial.ttf", size=x)
    draw.text((x, y), label_a, "black", font=font)
    draw.text((x, y + img_a.height), label_b, "black", font=font)

    return dst


def horizontal_concat(img_a, img_b):
    return horizontal_concat_with_labels(img_a, "A", img_b, "B")


def horizontal_concat_with_labels(img_a, label_a, img_b, label_b):
    dst = Image.new("RGB", (img_a.width + img_b.width, img_a.height))
    dst.paste(img_a, (0, 0))
    dst.paste(img_b, (img_a.width, 0))
    draw = ImageDraw.Draw(dst)
    x = img_a.width // 20
    y = img_a.height // 20
    font = ImageFont.truetype(font="Arial.ttf", size=x)
    draw.text((x, y), label_a, "black", font=font)
    draw.text((x + img_a.width, y), label_b, "black", font=font)

    return dst


def save_img(img_a, label_a, img_b, label_b, d_name, uuid):
    dpi = 300
    vertical_concat(img_a, img_b).save(
        ex_path.joinpath(
            f"data/exex/vertical_pictures/{d_name.replace('_','-')}/{uuid}.png"
        ),
        dpi=(dpi, dpi),
        quality=100,
    )
    vertical_concat_with_labels(
        img_a,
        label_a,
        img_b,
        label_b,
    ).save(
        ex_path.joinpath(
            f"data/exex/vertical_pictures_with_labels/{d_name.replace('_','-')}/{uuid}.png"
        ),
        dpi=(dpi, dpi),
        quality=100,
    )

    horizontal_concat(img_a, img_b).save(
        ex_path.joinpath(
            f"data/exex/horizontal_pictures/{d_name.replace('_','-')}/{uuid}.png"
        ),
        dpi=(dpi, dpi),
        quality=100,
    )
    horizontal_concat_with_labels(
        img_a,
        label_a,
        img_b,
        label_b,
    ).save(
        ex_path.joinpath(
            f"data/exex/horizontal_pictures_with_labels/{d_name.replace('_','-')}/{uuid}.png"
        ),
        dpi=(dpi, dpi),
        quality=100,
    )


for d_name in d_names:
    ex_path.joinpath(
        f"data/exex/vertical_pictures/{d_name.replace('_','-')}"
    ).mkdir(parents=True, exist_ok=True)
    ex_path.joinpath(
        f"data/exex/horizontal_pictures/{d_name.replace('_','-')}"
    ).mkdir(parents=True, exist_ok=True)
    ex_path.joinpath(
        f"data/exex/vertical_pictures_with_labels/{d_name.replace('_','-')}"
    ).mkdir(parents=True, exist_ok=True)
    ex_path.joinpath(
        f"data/exex/horizontal_pictures_with_labels/{d_name.replace('_','-')}"
    ).mkdir(parents=True, exist_ok=True)
export_data = {}
types = ["empirical", "multi", "single"]
for d_name in d_names:
    for d_seed in drawing_seeds:
        empirical = Image.open(
            ex_path.joinpath(
                f"data/exex/{d_name.replace('_','-')}/empirical-{d_seed}.png"
            )
        )
        multi = Image.open(
            ex_path.joinpath(
                f"data/exex/{d_name.replace('_','-')}/multi-{d_seed}.png"
            )
        )
        single = Image.open(
            ex_path.joinpath(
                f"data/exex/{d_name.replace('_','-')}/single-{d_seed}.png"
            )
        )
        empirical_multi = [
            {"img": empirical, "type": "empirical"},
            {"img": multi, "type": "multi"},
        ]
        empirical_single = [
            {"img": empirical, "type": "empirical"},
            {"img": single, "type": "single"},
        ]
        multi_single = [
            {"img": multi, "type": "multi"},
            {"img": single, "type": "single"},
        ]

        shuffle(empirical_multi)
        shuffle(empirical_single)
        shuffle(multi_single)

        uuid = str(uuid4())
        export_data[uuid] = {
            "id": uuid,
            "A": empirical_multi[0]["type"],
            "B": empirical_multi[1]["type"],
            "seed": d_seed,
            "dataset": d_name.replace("_", "-"),
        }
        save_img(
            empirical_multi[0]["img"],
            empirical_multi[0]["type"],
            empirical_multi[1]["img"],
            empirical_multi[1]["type"],
            d_name,
            uuid,
        )

        uuid = str(uuid4())
        export_data[uuid] = {
            "id": uuid,
            "A": empirical_single[0]["type"],
            "B": empirical_single[1]["type"],
            "seed": d_seed,
            "dataset": d_name.replace("_", "-"),
        }
        save_img(
            empirical_single[0]["img"],
            empirical_single[0]["type"],
            empirical_single[1]["img"],
            empirical_single[1]["type"],
            d_name,
            uuid,
        )

        uuid = str(uuid4())
        export_data[uuid] = {
            "id": uuid,
            "A": multi_single[0]["type"],
            "B": multi_single[1]["type"],
            "seed": d_seed,
            "dataset": d_name.replace("_", "-"),
        }
        save_img(
            multi_single[0]["img"],
            multi_single[0]["type"],
            multi_single[1]["img"],
            multi_single[1]["type"],
            d_name,
            uuid,
        )

with open(ex_path.joinpath("data/exex/pictures.json"), mode="w") as f:
    json.dump(export_data, f)